In [1]:
import pandas as pd
import mc_processor
import os

In [2]:
## toggle

# survey_year = input("What year is the survey you want to analyze from? (2022, 2023, or 2024)")
survey_year = str(2024)

In [3]:
# read results files
codebook = pd.read_csv(os.path.join(survey_year,'raw/levels_codebook.csv'), encoding='utf-8')
data = pd.read_csv(os.path.join(survey_year,'raw/data.csv'), encoding='utf-8')
q_codebook = pd.read_csv(os.path.join(survey_year,'raw/question_codebook.csv'), encoding='utf-8', index_col = 'qid')

# q_codebook into dict
q_codebook = q_codebook.to_dict().get('qidFull')

In [4]:
nulldata = pd.DataFrame(data.isnull().sum(),columns=["Sum_Null"]) ##2203 rows in entire dataset
nulldata.sort_values(by=["Sum_Null"],ascending=False,inplace=True)

# nulldata[nulldata["Sum_Null"]>0].head(20)

Use `get_percents` to get percent who selected each option, works for questions that have multiple or single selection.

Demo input optional. For all demo categories, see next cell.

In [5]:
# all demo categories
for key in q_codebook:
    if "dem" in key:
        print(key)

xdemAll
demZIP
xdemGender
demAgeGeneration
xdemIdeo3
xdemInc3
xdemWhite
xdemHispBin
demBlackBin
demRaceOther
xdemReligion
xdemReligOther
xdemEvang
xdemUsr
xdemEmploy
xdemMilHH1
xdemBidenApprove
xdemBidenApprove2
demInsured
demInsType
demUnion
demLGBTQ1
xdemMarried
BPCxdem1
BPCxdem2
BPCxdem3
BPCxdem4
BPCxdem5


In [19]:
# testing on range of questions

questions = []
for i in range(1,25):
    questions.append("BPC"+str(i))

for question in questions:
    
    #print question text
    for key in q_codebook:
        if key.startswith(question):
            # print(question)
            # print(q_codebook[key])
            if '---' in q_codebook[key]:
                print(f"\n\n{question}: {q_codebook[key].split('---')[0]}")
                break
            elif '\\\\' in q_codebook[key]:
                q_text = q_codebook[key].split('\\\\')[0]
                print(f"\n\n{question}: {q_text}")
                break
            else:
                print(f"\n\n{question}: {q_codebook[key]}")


    try:
        demo = None 
        # demo="xpid3"
        results = mc_processor.get_percents(data,codebook,q_codebook,question,demo) 

        if demo:
            # create demo directory
            if not os.path.exists(f"{survey_year}/processed/{demo}"):
                os.makedirs(f"{survey_year}/processed/{demo}")
            # save to demo directory
            results.to_csv(f"{survey_year}/processed/{demo}/{question}.csv")
        else:
            results.to_csv(f"{survey_year}/processed/{question}.csv")

        display(results.style.format("{:.0%}"))

    except Exception as e:
        print(f"*Issue with {question} {str(e)[:100]}\n")



BPC1: How often do you seek out the following kinds of information about elections, if at all? 


,"Only around major election dates or deadlines (e.g. Voter registration deadlines, Election Day)",A few times in the weeks around major election dates or deadlines,Regularly in the weeks around major election dates or deadlines,Consistently throughout the year,Never
Information about how to register and vote,23%,13%,14%,12%,39%
Information about how elections are run,20%,16%,15%,13%,36%
Information about who won an election,30%,13%,19%,14%,23%
Information about candidates,18%,18%,20%,22%,23%
News about campaigns,16%,18%,20%,19%,27%




BPC2: Below is a list of sources from which people might get news and information about elections. After reading this list of sources, please choose the three that you use most often to learn about elections. 


,overall
Social media,39%
National television,36%
Local or regional television,28%
Search through Google or other search engines,23%
"National news outlets, print or online",22%
"Local or state news outlets, print or online",19%
Radio,16%
Podcasts,10%
I am not interested in election news or information,8%
"News aggregator apps (e.g. NewsBreak, Smart News, Apple News)",8%




BPC3: If you wanted to know more about how elections are run in the United States, where are you most likely look for that information? Please select up to three. 


...

,overall
Social media platforms and government should share the responsibility,33%
Mainly the responsibility of social media platforms,22%
Don't know/ No opinion,20%
Mainly the responsibility of government,14%
Neither are responsible,11%




BPC19: Do you agree or disagree with the following statement?In the most recent election I voted in, my voting experience was mostly positive.


,overall
Strongly agree,36%
Somewhat agree,29%
Don't know/ No opinion,23%
Somewhat disagree,7%
Strongly disagree,4%




BPC20: How confident are you that your vote in the November 2024 General Election will be counted as you intend?


BPC20: How confident are you that your vote in the November 2024 General Election will be counted as you intend?


BPC20: How confident are you that your vote in the November 2024 General Election will be counted as you intend?
*Issue with BPC20 'BPC20'



BPC21: How confident are you that your votes in your county or city will be counted as intendedin the November 2024 General Election?


BPC21: How confident are you that your votes in your county or city will be counted as intendedin the November 2024 General Election?


BPC21: How confident are you that your votes in your county or city will be counted as intendedin the November 2024 General Election?
*Issue with BPC21 'BPC21'



BPC22: How confident are you that votes in your state will be counted as intended in the November 2024 General Election?


BPC22: How confident are you that votes in your state will be counte

,Very concerned,Somewhat concerned,Not too concerned,Not concerned at all,Don't know/ No opinion
Inaccurate or misleading information about elections,33%,29%,16%,8%,14%
Use of AI to spread disinformation or manipulate public opinion,35%,27%,14%,7%,16%
Interference from foreign entities (including countries),26%,30%,19%,10%,16%
"Counting ineligible votes (e.g. duplicate, non-citizen, or deceased voters)",25%,26%,19%,14%,15%
Attempts to overturn the results of a fair election,30%,27%,17%,10%,16%
Biased or inaccurate counting of ballots,26%,26%,20%,12%,15%
Illegal or improper use of mail-in ballots or drop boxes,27%,26%,18%,14%,15%
"Presence of guns, violence, or intimidation at voting locations",23%,26%,22%,14%,15%
Violence or civil unrest after election day,30%,30%,16%,10%,15%
Restrictive voting equipment,19%,25%,23%,14%,18%


Figuring out conjoint issue

In [7]:
# get columsn in data that start iwth "BPC6"
data[data.filter(regex='^BPC6').columns]

,BPC6_1NET,BPC6_2NET,BPC6_3NET,BPC6_4NET,BPC6_5NET,BPC6_6NET,BPC6_7NET,BPC6_8NET,BPC6_9NET,BPC6_10NET,BPC6_11NET,BPC6_12NET,BPC6_13NET,BPC6_14NET,BPC6_15NET,BPC6_16NET,BPC6_17NET,BPC6_18NET,BPC6_19NET,BPC6_20NET
0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1
1,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2
2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1
3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2
4,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2199,1,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,1,2,2,2
2200,2,1,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2,2,2
2201,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2
2202,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2


In [8]:
def data_type_check(data,q_columns):
    """
    returns boolean value indicating question type
    """
    matrix = False
    multiple_selections = False

    if len(q_columns) == 0: # underscore only used when each option has selections
        return multiple_selections, matrix # single selection

    ## to address single select questions with a short response text answer
    elif (len(q_columns) == 1) and ("TEXT" in q_columns[0]):
        return multiple_selections, matrix # single selection

    ## selects matrix quetsion (if BOTH multiple columns per question and multiple response categories within each column)
    elif (len(q_columns) > 1) and (len(data[[q_columns[0]]].dropna()[q_columns[0]].unique()) > 2):
        matrix = True
        return multiple_selections, matrix

    ## otherwise, probably mulitple selections
    else:
        multiple_selections = True
        return multiple_selections, matrix

In [9]:
q_columns = data.filter(regex='^'+question+"_").columns

data_type_check(data,q_columns)

data.filter(regex='^' + question + r'[a-z]$').columns

Index([], dtype='object')